# --- Zomato SQL Poject ----


# Project Objective

The project involved analyzing customer behavior on the Zomato platform, which is a food delivery, sales, and customer. The data used for the analysis was about Zomato sales based on membership, which contained information about sales, customers, products and membership.

# Project Approach

We approached the project first and had to clean the data to ensure that it was in a usable format. Checked any missing values and dealing with any data quality issues. Then used SQL queries to extract the data from the Zomato dataset and performed aggregate calculations on the data to answer the questions.

# Provided Task 

In [ ]:
 1. What is the total amount each customer spent on zomato?
    
    SELECT S.userid, SUM(p.price) As Total_Amount_Spent 
    from sales S 
    join product p on S.product_id = P.product_id 
    GROUP BY S.userid;

In [ ]:
2. How many days has each customer visited zomato?

   SELECT userid, COUNT (DISTINCT created_date) AS Visited_Count 
   from sales 
   GROUP BY userid ;


In [ ]:
3. What was the first product purchased by each of the customer?

   SELECT * 
   from (SELECT * 
          ,RANK() OVER(PARTITION by userid ORDER by created_date) as Ranking 
         from sales)s 
   where Ranking = 1 ;

In [ ]:
4. What is the most purchased item on the menu and how many times was it purchased by all customers?

    SELECT userid, COUNT(product_id) As Purchased_count 
    from sales 
    where product_id =(SELECT product_id 
                       from sales 
                       group by product_id ORDER by COUNT(product_id) DESC LIMIT 1) 
    group by userid ;

In [ ]:
5.  Which item was the most popular for each customer?

    SELECT * 
    from (SELECT *,rank() over (PARTITION by userid ORDER  by product_count DESC) AS RANKING 
          from(SELECT userid , product_id , count(product_id) As product_count 
               from sales 
               group by userid, product_id order by count(product_id) DESC) a )b
    where RANKING = 1 ;

In [ ]:
6. Which item was purchased first by customer after they become a member ?
  
   SELECT * 
   FROM (SELECT m.*,rank() over(partition by userid order by created_date) ranking 
         from(SELECT s.userid,s.created_date,s.product_id,g.gold_signup_date 
              FROM sales As s INNER JOIN goldusers_signup AS g on s.userid=g.userid and created_date >= gold_signup_date) m)n
   where ranking = 1 ;

In [ ]:
7. Which item was purchased just before the customer became a member?

   SELECT * 
   FROM (SELECT m.*,rank() over(partition by userid order by created_date) ranking 
         from(SELECT s.userid,s.created_date,s.product_id,g.gold_signup_date 
              FROM sales As s INNER JOIN goldusers_signup AS g on s.userid=g.userid and created_date <= gold_signup_date) m)n
   where ranking = 1 ;

In [ ]:
8. what is total orders and amount spent for each member before they become a member?

   SELECT userid, COUNT(created_date) Order_purchased, SUM(price) total_amount_spent 
   from(SELECT b.*, c.price 
        FROM(SELECT s.userid,s.created_date,s.product_id,g.gold_signup_date 
             FROM sales As s INNER JOIN goldusers_signup AS g on s.userid=g.userid and created_date <= gold_signup_date) b    
             INNER JOIN Product c ON b.product_id=c.product_id) u 
   GROUP By userid ;


In [ ]:
9. Ranking all the transaction of the customers 

     SELECT *
      , RANK() OVER(PARTITION BY userid ORDER BY created_date) AS RANKING  
     from sales ;

In [ ]:
10. Ranking all transaction for each member whenever they are zomato goldmember for every non goldmember transaction mark as NA 

    SELECT t.*
     , case when RANKING = 0 THEN 'NA' ELSE RANKING END AS RNK_TRANSACTION 
     FROM (select e.*
           , CAST((CASE WHEN gold_signup_date IS NULL THEN 0 ELSE RANK() OVER(PARTITION BY userid ORDER BY created_date DESC)
                   END) AS VARCHAR) AS RANKING 
           FROM (SELECT s.userid,s.created_date,s.product_id,g.gold_signup_date 
                 FROM sales As s LEFT JOIN goldusers_signup AS g on s.userid=g.userid and created_date >= gold_signup_date)e)t

